# BerryBox Berry Segmentation

## Run inferencing using a trained YOLOv8 model

This notebook is meant to be run on a Windows PC using a CPU

In [ ]:
# Open ultralytics
import ultralytics
ultralytics.checks()

# Set project directory and change directory
import os
import shutil
proj_dir = os.getcwd()
proj_dir
%ls

# Model inference size
image_size = (1856, 2784)

1. Import a YOLO model and export it using OpenVINO

In [2]:
# Set the path to the model
# model_path = os.path.join(proj_dir, "models/berrybox_inst_seg_nano_best_20240324.pt")
model_path = "C:/Users/jeffrey.neyhart/Documents/Phenomics/berryboxai/models/berrybox_inst_seg_nano_best_20240324.pt"
# OpenVino model path
ov_model_path = model_path.replace(".pt", "_openvino_model")

# Attempt to find the openvino version of the model;
# If it does not exist, export the model
if not os.path.exists(ov_model_path):

    os.chdir(os.path.dirname(model_path))

    # Copy the model to the OpenVino location 
    model_path_local = model_path

    # Load the model with YOLO
    from ultralytics import YOLO
    model = YOLO(model_path_local)

    # Export the model using openVINO
    model.export(format = "openvino", imgsz = image_size, half = True)

    os.chdir(proj_dir)

2. Import dependecies, set parameters
Note: Make sure functions.py is in the current working directory

In [5]:
from functions import * # load all functions
from ultralytics import YOLO
import os
import torch
import gc
import shutil

gc.collect()   # collect garbage

device = '0' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')


"""
------------------------------------------------------------------------------------
Set Directories
------------------------------------------------------------------------------------
"""
model_dir = ov_model_path # path to the model
image_dir = 'images' # path to the image folder
save_dir = 'output' # path to save the results

# shutil.rmtree(save_dir, ignore_errors=True)

"""
------------------------------------------------------------------------------------
Set Model Parameters (you can change these parameters to suit your needs)
------------------------------------------------------------------------------------
"""
model_params = {
    'project': save_dir, # project name
    'name': "berrybox_" + os.path.basename(proj_dir), # run name
    'save': False, # save image results
    'show_labels': True,   # hide labels
    'show_conf': True, # hide confidences
    'save_crop': False, # save cropped prediction boxes
    'line_width': 3, # bounding box line width
    'conf': 0.70, # confidence threshold
    'iou': 0.75, # NMS IoU threshold
    'imgsz': image_size,
    'exist_ok': True, # if True, it overwrites current 'name' saving folder
    'half': True, # use FP16 half-precision inference True/False
    'cache': False, # use cache images for faster inference
    'retina_masks': False, #use high resolution seg mask
    'device': device, # cuda device, i.e. 0 or 0,1,2,3 or cpu
    'verbose': True
}

# Load the model
model = YOLO(model_dir, task = "segment")


Using device: cpu


3. Run Inference

In [ ]:
out = run_inference(input_dir = image_dir, output_dir = save_dir, params = model_params, model = model)

gc.collect()    